# Google Gemini using Langchain

In [1]:
import sys, os
# import google.generativeai as genai
sys.path.append(os.path.abspath(os.path.join('.', 'secret')))
from secret_info import google_genai_api

GOGGLE_API_KEY = google_genai_api
# genai.configure(api_key = GOGGLE_API_KEY)

## LLMs wrappers

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory
chat = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7, safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },)
chat.invoke("explain how a car works in one sentence").content

'A car uses an engine to convert gasoline into motion, which is then transferred through a transmission to the wheels, allowing the vehicle to move.'

In [3]:
# Same as above
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7)
llm.invoke("explain large language models in one sentence")

'Large language models (LLMs) are AI systems trained on massive text datasets that allow for advanced natural language processing tasks such as text generation, translation, question answering, and dialogue.'

## CHAT MODEL

In [4]:
from langchain.schema import HumanMessage, SystemMessage

chat = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
]
response = chat.invoke(input=messages)
print(response.content,end='\n')

```python
import numpy as np
import tensorflow as tf

# Generate simulated data
x_train = np.random.rand(100, 10)
y_train = np.random.randint(2, size=(100, 1))

# Define the neural network model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation='relu', input_shape=(10,)),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10)

# Evaluate the model
loss, accuracy = model.evaluate(x_train, y_train)
print(f"Loss: {loss}, Accuracy: {accuracy}")
```


## PROMPTS

### Single Prompt Interaction 

In [5]:
# Single promt interaction: 
from langchain import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import SimpleSequentialChain

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY)
# First prompt
template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in 200 words
"""
prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)
chain = prompt | llm
chain.invoke({'concept': 'car'})

'A car is a wheeled vehicle that is used for transportation. It is typically powered by an engine, and has four wheels. Cars can be used to transport people or goods, and can be either privately owned or rented.\n\nCars have been around for over a century, and have undergone significant changes over the years. Early cars were powered by steam engines, and were very slow and unreliable. However, as technology improved, cars became more powerful and reliable. In the early 20th century, the internal combustion engine was invented, which made cars much more efficient and powerful.\n\nToday, cars are an essential part of modern life. They are used for a wide variety of purposes, and are available in a wide range of styles and prices. Cars have made it possible for people to travel more easily and quickly, and have helped to connect people all over the world.'

### Multi Prompt Interactions 

This portion uses LCEL as SimpleSequentialChain and LLChains are now deprecated, and generate an error. See: 

https://stackoverflow.com/questions/78508462/keyerror-in-simplesequentialchain/78562683#78562683

https://github.com/langchain-ai/langchain/discussions/20137

In [6]:
# LCEL
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY)

template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in 200 words
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

second_prompt = PromptTemplate(
    input_variables=["structure"],
    template="Turn the concept description of {structure} and explain it to me like I'm five in 500 words",
)
initial_chain = prompt | llm | StrOutputParser()
review_chain = second_prompt | llm | StrOutputParser()

chain = ({"structure" : initial_chain} 
         | RunnablePassthrough.assign(review=review_chain)
        )
result = chain.invoke({"concept": 'car'})
result

{'structure': "A car is a complex machine that requires a deep understanding of various scientific and engineering disciplines to fully comprehend. At its core, a car is a mode of transportation designed to move people or goods from one place to another. It consists of several key components that work together to provide mobility, including an engine, transmission, wheels, and a chassis.\n\nThe engine is responsible for converting fuel into energy, which is then used to power the wheels and propel the car forward. The transmission allows the driver to change gears, which adjusts the ratio of engine speed to wheel speed, enabling the car to operate efficiently at different speeds and loads. The wheels provide traction and support the weight of the car, while the chassis serves as the structural framework that holds all the components together.\n\nIn addition to these core components, cars also incorporate a wide range of advanced technologies, including computer systems, sensors, and ac

In [7]:
# This example generates as error 
from langchain import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain.chains import SimpleSequentialChain

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY)
# First prompt
template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in 200 words
"""
prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)
chain = prompt | llm

# Define a second prompt 
second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words",
)
chain_two = second_prompt | llm

overall_chain = SimpleSequentialChain(chains=[chain, chain_two])
# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("autoencoder")
print(explanation)

KeyError: 'chains'

## Pandas DATAFRAME

https://python.langchain.com/v0.1/docs/integrations/toolkits/pandas/

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7)